In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%matplotlib notebook
%autosave 0
import numpy as np
import matplotlib.pyplot as plt
import torch
import pyro

import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Information Theory

> What is information? Can we measure it?

Information Theory is the mathematical study of the quantification and transmission of information proposed by **Claude Shannon** on this seminal work: *A Mathematical Theory of Communication*, 1948

Shannon considered the output of a noisy source as a random variable $X$ taking $M$ possible values $\mathcal{A} = \{x_1, x_2, x_3, \ldots, x_M\}$

Each value $x_i$ have an associated probability $P(X=x_i) = p_i$

> What is the amount of information carried by $x_i$?

Shannon defined the amount of information as

$$
I(x_i) = \log_2 \frac{1}{p_i},
$$

which is measured in **bits**

> One bit is the amount of information needed to choose between two **equiprobable** states



#### Example: A meteorological station that sends tomorrow's weather prediction

The dictionary of messages: (1) Rainy, (2) Cloudy, (3) Partially cloudy, (4) Sunny

Their probabilities are: $p_1=1/2$, $p_2=1/4$, $p_3=1/8$, $p_4=1/8$

The minimum number of yes/no questions (equiprobable) needed to guess tomorrow's weather:

- Is it going to rain? 
- No: Is it going to be cloudy?
- No: Is it going to be sunny?

Amount of information:
- Rainy: $\log_2 \frac{1}{p_1} = \log_2 2 = 1$ bits
- Cloudy: $2$ bits 
- Partially cloudy and Sunny: $3$ bits

> The larger the probability the smallest information it carries

> Amount of information is also called surprise

## Shannon's entropy

After defining the amount of information for a state Shannon's defined the average information of the source $X$ as

$$
\begin{align}
H(X) &= \mathbb{E}_{x\sim X}\left [\log_2 \frac{1}{P(x)} \right] \nonumber \\
&= - \sum_{x\in \mathcal{A}} P(x) \log_2 P(X)  \nonumber \\
&= - \sum_{i=1}^M p_i \log_2 p_i  ~ \text{[bits]} \nonumber
\end{align}
$$

and called it the **entropy** of the source

> Entropy is the "average information of the source"

#### Properties:
- Entropy is nonnegative: $H(X)>0$
- Entropy is equal to zero when $p_j = 1 \wedge p_i = 0, i \neq j$
- Entropy is maximum when $X$ is uniformly distributed $p_i = \frac{1}{M}$, $H(X) = \log_2(M)$

> The more random the source is the larger its entropy

Differential entropy for continuous variables as 

$$
H(p) = - \int p(x) \log p(x) \,dx ~ \text{[nats]}
$$

where $p(x)$ is the probability density function (pdf) of $X$

## Relative Entropy: [Kullback](https://en.wikipedia.org/wiki/Solomon_Kullback)-[Leibler](https://en.wikipedia.org/wiki/Richard_Leibler) divergence

Consider a continuous random variable $X$ and two distributions $q(x)$ and $p(x)$ defined on its probability space

The relative entropy between these distributions is 
$$
\begin{align}
D_{\text{KL}} \left [ p(x) || q(x) \right] &= \mathbb{E}_{x \sim p(x)} \left [ \log \frac{p(x)}{q(x)} \right ] \nonumber \\
&= \mathbb{E}_{x \sim p(x)} \left [ \log p(x) \right ]  - \mathbb{E}_{x \sim p(x)} \left [ \log q(x) \right ],  \nonumber \\
&= \int p(x) \log p(x) \,dx  - \int p(x) \log q(x) \,dx  \nonumber 
\end{align}
$$
which is also known as the **Kullback-Leibler divergence**

- The left hand side term is the negative entropy of p(x)
- The right hand side term is called the **cross-entropy of q(x) relative to p(x)** 

#### Intepretations of KL
- Coding: Expected number of "extra bits" needed to code p(x) using a code optimal for q(x)
- Bayesian modeling: Amount of information lost when q(x) is used as a model for p(x)

#### Non-negativity

The KL divergence is non-negative
$$
D_{\text{KL}} \left [ p(x) || q(x) \right] \geq 0
$$
with the equality holding for $p(x) \equiv q(x)$

This is given by the [Gibbs inequality](https://en.wikipedia.org/wiki/Gibbs%27_inequality)

$$
- \int p(x) \log p(x) \,dx  \leq - \int p(x) \log q(x) \,dx 
$$

> then entropy of $p(x)$ is equal or less than the cross-entropy of $q(x)$ relative to $p(x)$



#### Relation with mutual information

The KL is related to the mutual information between random variables as

$$
\text{MI}(X, Y) = D_{\text{KL}} \left [ p(x, y) || p(x)p(y) \right]
$$

#### Asymmetry

The KL divergence is asymmetric
$$
D_{\text{KL}} \left [ p(x) || q(x) \right] \neq D_{\text{KL}} \left [ q(x) || p(x) \right]
$$

- The KL is not a proper distance (no triangle inequility either)
- Forward and Reverse KL have different meanings (we will explore them soon)

# Probabilistic generative models

Let's say we have $N$ continuous observations 

$$
\mathcal{D} = \{x_1, x_2, \ldots, x_N\}
$$ 

(Assuming that they are iid) there is unknown distribution that generated these samples

$$
x_i \sim p^*(x)
$$

The goal of **generative modeling** is to learn a probabilistic model 

$$
p_\theta(x)
$$ 

with parameters $\theta~$ that "mimics" $p^*(x)$

In a few words:

> match  $p_\theta (x)$ to $p^*(x)$

After matching $p_\theta(x)$ we can use it to sample new data: **generation**

Later we will extend this definition to **joint distributions**

### Recipe for fitting a generative model


1. Select a parametric form for $p_\theta (x)$
1. Write the difference between $p_\theta (x)$  and $p^*(x)$
1. Minimize this difference as a function of $\theta~$

> How do we compute the difference between probability distributions?

We can use the **forward** KL divergence

$$
\begin{align}
\hat \theta &= \text{arg} \min_\theta D_{\text{KL}} \left [ p^*(x) || p_\theta(x) \right]  \nonumber \\
&= \text{arg} \min_\theta \mathbb{E}_{x \sim p^*(x)} \left [ \log p^*(x) \right ]  - \mathbb{E}_{x \sim p^*(x)} \left [ \log p_\theta(x) \right ] \nonumber \\
& = \text{arg} \max_\theta \mathbb{E}_{x \sim p^*(x)} \left [ \log p_\theta(x) \right ]
\end{align}
$$

- Bad news: We can't evaluate $\mathbb{E}_{x \sim p^*(x)} \left [ \log p^*(x) \right ]$ 
- Good news: It doesn't depend on $\theta~$, we can drop it


### Relation with Maximum Likelihood

We found that

$$
\min_\theta D_{\text{KL}} \left [ p^*(x) || p_\theta(x) \right] = \max_\theta\mathbb{E}_{x \sim p^*(x)} \left [ \log p_\theta(x) \right ]
$$

If we approximate the expected value with an average over our finite dataset
$$
\mathbb{E}_{x \sim p^*(x)} \left [ \log p_\theta(x) \right ] \approx \sum_{i=1}^N \log p_\theta(x_i)
$$

We get the log likelihood of $\theta~$!

> Minimizing the forward KL divegence $\equiv$ Maximizing the log likelihood of the model

#### Example: Fitting a univariate Gaussian model

We propose $p_\theta(x) = \mathcal{N}(x|\mu, \sigma^2)$

The parameters are $\theta=(\mu, \sigma)$

The log likelihood is 

$$
\mathbb{E}_{x \sim p^*(x)} \left [ \log p_\theta(x) \right ] \approx  \sum_{i=1}^N \log p_\theta(x_i) =  -\frac{N}{2}\log(2\pi\sigma^2) - \frac{1}{2\sigma^2}  \sum_{i=1}^N (x_i - \mu)^2
$$

The derivatives are

$$
\frac{d}{d\mu} \sum_{i=1}^N \log p_\theta(x_i) = \frac{1}{\sigma^2}  \sum_{i=1}^N (x_i - \mu)
$$
and 
$$
\frac{d}{d\sigma^2} \sum_{i=1}^N \log p_\theta(x_i) = -\frac{N}{2\sigma^2} +  \frac{1}{\sigma^4}  \sum_{i=1}^N (x_i - \mu)^2
$$

By setting them to zero we obtain the analytical MLE of $\mu$ and $\sigma$

$$
\hat \mu = \frac{1}{N} \sum_{i=1}^N x_i
$$
$$
\hat \sigma^2 = \frac{1}{N} \sum_{i=1}^N (x_i - \hat \mu)^2
$$

In [ ]:
data = 5 + 2*np.random.randn(1000) # N(5, sqrt(2))

fig, ax = plt.subplots()
ax.hist(data, bins=20, density=True);

## Additional material

- Daniel Commenges, ["Information Theory and Statistics: an overview"](https://arxiv.org/pdf/1511.00860.pdf)

https://blog.evjang.com/2016/08/variational-bayes.html

https://www.inference.vc/maximum-likelihood-for-representation-learning-2/

https://www.tuananhle.co.uk/notes/reverse-forward-kl.html

https://wiseodd.github.io/techblog/2016/12/21/forward-reverse-kl/

https://dibyaghosh.com/blog/probability/kldivergence.html

A Tutorial on Deep Probabilistic Generative Models Ryan addams